In [ ]:

# Description: This file contains functions for analyzing neural data, specifically for mouse and monkey datasets
import time
import pickle
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)


import os
results_dir = os.path.join(project_root,'results/fig_3/')


# ensure the results directory exists
os.makedirs(results_dir, exist_ok=True)
os.chdir(project_root)

import sys
sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)
from utils.LFPs_timelags_functions import make_df_timelags

save_figs = False
fig_dir = main_dir + f'/results/paper_figures/revisions/'

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, Normalize
from utils.fig_7_functions import get_refons_refoffs

# Define your custom colormap colors (e.g., dark blue to light blue)
colors = ['white', '#55C0B7', '#054C47']
# Normalize the colormap between 0 and 1 (adjust as needed for your data)
norm = Normalize(vmin=0, vmax=1)
# Create the custom colormap
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)


colors = ['white', '#F58B8E', '#94221F']
# Create the custom colormap
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)



In [ ]:
monkey_names = ['L','A']
all_monkey_stats = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_7',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats[monkey_name] = pickle.load(f)

### monkey L

In [ ]:
monkey_name = "L"
monkey_stats = all_monkey_stats[monkey_name]


In [ ]:

ref_area='V4'
ref_duration=200
w_size=10
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v4_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)


In [ ]:
BANDS = {
    "low_2_12":   (2, 12),
    "beta_12_30": (12, 30),
    "gamma_30_45":(30, 45),
    "hgamma_55_95":(55, 95),
}
fontsize=6
timelag_plot_args = {
	'figsize': (1.8,2),
	'tick_params':{'axis':'both','labelsize':fontsize,'width':0.3,'length':2},
	'sup_title_args':{'y':1.05,'fontsize':fontsize*1.2},
	'supplots_adjust':{'wspace':0.1,'hspace':0.55},
	'ax_title_args':{'fontsize':fontsize*1.1,'y':0.97},
}

In [ ]:
# Fig Supp 13A
ref_area='V4'
x_tick_labels = [ 'Low\n(2-12Hz)', 'Beta\n(12-30Hz)', 'Gamma\n(30-45Hz)', 'High Gamma\n(55-95Hz)']
title_band_dict = {band: label for band, label in zip(BANDS.keys(), x_tick_labels)}
fig, axes = plt.subplots(2,2, figsize=(1.8,2), sharey=True, sharex=True)
mappable = None
for a, (ax, band) in enumerate(zip(axes.flatten(),BANDS.keys())):
	df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)&(df_v4_['Band']==band)]
	# max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
	max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Neuron','Permutation'])['EV'].idxmax()
	heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	data =heatmap_data_max/heatmap_data_all
	data = data.reindex(df_v4.Ref_Times.unique())
	assert np.allclose(data.sum(axis=1).values, 1.0, atol=1e-6)	
	hm = sns.heatmap(data, cmap=v4_cmap, vmin=0, vmax=0.8, ax=ax, cbar=False)
	if mappable is None:
		mappable = hm.collections[0]    # save one handle for the shared cbar
	ax.tick_params(axis='both', labelsize=6, pad=1, width=0.3, length=2)
	if a==0 or a==2:
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel('')
	if a==2 or a==3:
		ax.set_xlabel('predictor window\noffset (ms)', fontsize=6)
	else:
		ax.set_xlabel('')
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	# ax.set_xticks([0.5,2.5,4.4,6.5,8.5], labels=[-100,-50,0,50,100])
	ax.set_title(f'{title_band_dict[band]}', fontsize=6*1.1, y=0.97)
	# draw a dotted line at the 0-ms column (if present)
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)

cax = fig.add_axes([0.92, 0.12, 0.03, 0.75])             # x, y, w, h in [0,1] fig coords
cbar = fig.colorbar(mappable, cax=cax)
cbar.outline.set_linewidth(0.5)
cbar.ax.tick_params(labelsize=6)
cbar.ax.tick_params(labelsize=6)
plt.subplots_adjust(wspace=0.1, hspace=0.55)
plt.suptitle(f'{predictor_area}→{ref_area}', fontsize=6*1.2, y=1.05)
if save_figs is True:
	plt.savefig(fig_dir + f'additional_questions_bands_on_monkey_timelag_percents_snr_V4_all_bands_10ms_wsize.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
ref_area='V1'
ref_duration=200
w_size=10
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v1_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)


In [ ]:
# Fig Supp 13D

ref_area='V1'

fig, axes = plt.subplots(2,2, figsize=(1.8,2), sharey=True, sharex=True)
mappable = None
for a, (ax, band) in enumerate(zip(axes.flatten(),BANDS.keys())):
	df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)&(df_v1_['Band']==band)]
	max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r', 'Neuron', 'Permutation'])['EV'].idxmax()
	heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	data =heatmap_data_max/heatmap_data_all
	data = data.reindex(df_v1.Ref_Times.unique())

	hm = sns.heatmap(data, cmap=v1_cmap, vmin=0, vmax=0.8, ax=ax, cbar=False)
	if mappable is None:
		mappable = hm.collections[0]    # save one handle for the shared cbar
	ax.tick_params(axis='both', labelsize=6, pad=1, width=0.3, length=2)
	if a==0 or a==2:
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel('')
		# ax.set_yticklabels([])
	if a==2 or a==3:
		ax.set_xlabel('predictor time\noffset (ms)', fontsize=6)
	else:
		ax.set_xlabel('')
		# ax.set_xticklabels([])
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	# ax.set_xticks([0.5,2.5,4.4,6.5,8.5], labels=[-100,-50,0,50,100])
	ax.set_title(f'{title_band_dict[band]}', fontsize=6, y=0.97)
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
cax = fig.add_axes([0.92, 0.12, 0.03, 0.75])             # x, y, w, h in [0,1] fig coords
cbar = fig.colorbar(mappable, cax=cax)
cbar.outline.set_linewidth(0.5)
cbar.ax.tick_params(labelsize=6)
cbar.ax.tick_params(labelsize=6)
plt.subplots_adjust(wspace=0.1, hspace=0.55)
plt.suptitle(f'{predictor_area}→{ref_area}', fontsize=6*1.2, y=1.05)
if save_figs is True:
	plt.savefig(fig_dir + f'additional_questions_bands_on_monkey_timelag_percents_snr_V1_all_bands_10ms_wsize.pdf', transparent=True, bbox_inches='tight')
plt.show()

### monkey A

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]

ref_area='V4'
ref_duration=200
w_size=10
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v4_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)


In [ ]:
#Fig Supp 13B

ref_area='V4'
x_tick_labels = [ 'Low\n(2-12Hz)', 'Beta\n(12-30Hz)', 'Gamma\n(30-45Hz)', 'High Gamma\n(55-95Hz)']
title_band_dict = {band: label for band, label in zip(BANDS.keys(), x_tick_labels)}
fig, axes = plt.subplots(2,2, figsize=(timelag_plot_args['figsize'][0],timelag_plot_args['figsize'][1]), sharey=True, sharex=True)
mappable = None
for a, (ax, band) in enumerate(zip(axes.flatten(),BANDS.keys())):
	df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)&(df_v4_['Band']==band)]
	# max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
	max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Neuron','Permutation'])['EV'].idxmax()
	heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	data =heatmap_data_max/heatmap_data_all
	data = data.reindex(df_v4.Ref_Times.unique())
	assert np.allclose(data.sum(axis=1).values, 1.0, atol=1e-6)	
	hm = sns.heatmap(data, cmap=v4_cmap, vmin=0, vmax=0.8, ax=ax, cbar=False)
	if mappable is None:
		mappable = hm.collections[0]    # save one handle for the shared cbar
	ax.tick_params(**timelag_plot_args['tick_params'])
	if a==0 or a==2:
		ax.set_ylabel('time window', fontsize=fontsize, labelpad=0)
	else:
		ax.set_ylabel('')
	if a==2 or a==3:
		ax.set_xlabel('predictor time\noffset (ms)', fontsize=fontsize)
	else:
		ax.set_xlabel('')
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	# ax.set_xticks([0.5,2.5,4.4,6.5,8.5], labels=[-100,-50,0,50,100])
	ax.set_title(f'{title_band_dict[band]}', **timelag_plot_args['ax_title_args'])
	# draw a dotted line at the 0-ms column (if present)
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)

cax = fig.add_axes([0.92, 0.12, 0.03, 0.75])             # x, y, w, h in [0,1] fig coords
cbar = fig.colorbar(mappable, cax=cax)
cbar.outline.set_linewidth(0.5)
cbar.ax.tick_params(labelsize=6)
cbar.ax.tick_params(labelsize=6)
plt.subplots_adjust(**timelag_plot_args['supplots_adjust'])
plt.suptitle(f'{predictor_area}→{ref_area}', **timelag_plot_args['sup_title_args'])
if save_figs is True:
	plt.savefig(fig_dir + f'additional_questions_bands_on_monkey_{monkey_name}_timelag_percents_snr_V4_all_bands_10ms_wsize.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:

ref_area='V1'
ref_duration=200
w_size=10
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v1_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)


In [ ]:
#Fig Supp 13E
ref_area='V1'

fig, axes = plt.subplots(2,2, figsize=(timelag_plot_args['figsize'][0],timelag_plot_args['figsize'][1]), sharey=True, sharex=True)
mappable = None
for a, (ax, band) in enumerate(zip(axes.flatten(),BANDS.keys())):
	df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)&(df_v1_['Band']==band)]
	max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r', 'Neuron', 'Permutation'])['EV'].idxmax()
	heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
	data =heatmap_data_max/heatmap_data_all
	data = data.reindex(df_v1.Ref_Times.unique())

	hm = sns.heatmap(data, cmap=v1_cmap, vmin=0, vmax=0.8, ax=ax, cbar=False)
	if mappable is None:
		mappable = hm.collections[0]    # save one handle for the shared cbar
	ax.tick_params(**timelag_plot_args['tick_params'])
	if a==0 or a==2:
		ax.set_ylabel('time window', fontsize=fontsize, labelpad=0)
	else:
		ax.set_ylabel('')
		# ax.set_yticklabels([])
	if a==2 or a==3:
		ax.set_xlabel('predictor time\noffset (ms)', fontsize=fontsize)
	else:
		ax.set_xlabel('')
		# ax.set_xticklabels([])
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	# ax.set_xticks([0.5,2.5,4.4,6.5,8.5], labels=[-100,-50,0,50,100])
	ax.set_title(f'{title_band_dict[band]}', **timelag_plot_args['ax_title_args'])
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
cax = fig.add_axes([0.92, 0.12, 0.03, 0.75])             # x, y, w, h in [0,1] fig coords
cbar = fig.colorbar(mappable, cax=cax)
cbar.outline.set_linewidth(0.5)
cbar.ax.tick_params(labelsize=6)
cbar.ax.tick_params(labelsize=6)
plt.subplots_adjust(**timelag_plot_args['supplots_adjust'])
plt.suptitle(f'{predictor_area}→{ref_area}', **timelag_plot_args['sup_title_args'])
if save_figs is True:
	plt.savefig(fig_dir + f'additional_questions_bands_on_monkey_{monkey_name}_timelag_percents_snr_V1_all_bands_10ms_wsize.pdf', transparent=True, bbox_inches='tight')
plt.show()

### Monkey $L_A$

In [ ]:

with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'A'

ref_area='V4'
x_tick_labels = [ 'Low\n(2-12Hz)', 'Beta\n(12-30Hz)', 'Gamma\n(30-45Hz)', 'High Gamma\n(55-95Hz)']
title_band_dict = {band: label for band, label in zip(BANDS.keys(), x_tick_labels)}
# we are going to loop through all seeds, and collect the variable data per area. after that we will average across seeds.
data_v4_all_seeds = {band: [] for band in BANDS.keys()}


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)

	ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
	df_v4_ = make_df_timelags(subsampled_monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)
	
	for a, band in enumerate(BANDS.keys()):
		df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)&(df_v4_['Band']==band)]
		max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Neuron','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v4.Ref_Times.unique())
		data_v4_all_seeds[band].append(data)
		


In [ ]:
# Fig Supp 13C

fig, axes = plt.subplots(2,2, figsize=(timelag_plot_args['figsize'][0],timelag_plot_args['figsize'][1]), sharey=True, sharex=True)
mappable = None
for a, (ax, band) in enumerate(zip(axes.flatten(),BANDS.keys())):
	data_band = pd.concat(data_v4_all_seeds[band], axis=0)
	mean_data_band = data_band.groupby(data_band.index).mean()
	data = mean_data_band.reindex(df_v4_.Ref_Times.unique())
	hm = sns.heatmap(data, cmap=v4_cmap, vmin=0, vmax=0.8, ax=ax, cbar=False)
	if mappable is None:
		mappable = hm.collections[0]    # save one handle for the shared cbar
	ax.tick_params(**timelag_plot_args['tick_params'])
	if a==0 or a==2:
		ax.set_ylabel('time window', fontsize=fontsize, labelpad=0)
	else:
		ax.set_ylabel('')
	if a==2 or a==3:
		ax.set_xlabel('predictor time\noffset (ms)', fontsize=fontsize)
	else:
		ax.set_xlabel('')
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	ax.set_title(f'{title_band_dict[band]}', **timelag_plot_args['ax_title_args'])
	# draw a dotted line at the 0-ms column (if present)
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)

cax = fig.add_axes([0.92, 0.12, 0.03, 0.75])             # x, y, w, h in [0,1] fig coords
cbar = fig.colorbar(mappable, cax=cax)
cbar.outline.set_linewidth(0.5)
cbar.ax.tick_params(labelsize=6)
cbar.ax.tick_params(labelsize=6)
plt.subplots_adjust(**timelag_plot_args['supplots_adjust'])
plt.suptitle(f'{predictor_area}→{ref_area}', **timelag_plot_args['sup_title_args'])
if save_figs is True:
	plt.savefig(fig_dir + f'additional_questions_bands_on_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_timelag_percents_snr_V4_all_bands_10ms_wsize_avgacrossseeds.pdf', transparent=True, bbox_inches='tight')


In [ ]:

ref_area='V1'
ref_duration=200
w_size=10
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)


with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'A'


x_tick_labels = [ 'Low\n(2-12Hz)', 'Beta\n(12-30Hz)', 'Gamma\n(30-45Hz)', 'High Gamma\n(55-95Hz)']
title_band_dict = {band: label for band, label in zip(BANDS.keys(), x_tick_labels)}
# we are going to loop through all seeds, and collect the variable data per area. after that we will average across seeds.
data_v1_all_seeds = {band: [] for band in BANDS.keys()}

for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	df_v1_ = make_df_timelags(subsampled_monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)

	for a, band in enumerate(BANDS.keys()):
		df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)&(df_v1_['Band']==band)]
		max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r', 'Neuron', 'Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v1.Ref_Times.unique())
		data_v1_all_seeds[band].append(data)

In [ ]:
# Fig Supp 13F

fig, axes = plt.subplots(2,2, figsize=(timelag_plot_args['figsize'][0],timelag_plot_args['figsize'][1]), sharey=True, sharex=True)
mappable = None
for a, (ax, band) in enumerate(zip(axes.flatten(),BANDS.keys())):
	data_band = pd.concat(data_v1_all_seeds[band], axis=0)
	mean_data_band = data_band.groupby(data_band.index).mean()
	data = mean_data_band.reindex(df_v1_.Ref_Times.unique())
	hm = sns.heatmap(data, cmap=v1_cmap, vmin=0, vmax=0.8, ax=ax, cbar=False)
	if mappable is None:
		mappable = hm.collections[0]    # save one handle for the shared cbar
	ax.tick_params(**timelag_plot_args['tick_params'])
	if a==0 or a==2:
		ax.set_ylabel('time window', fontsize=fontsize, labelpad=0)
	else:
		ax.set_ylabel('')
		# ax.set_yticklabels([])
	if a==2 or a==3:
		ax.set_xlabel('predictor time\noffset (ms)', fontsize=fontsize)
	else:
		ax.set_xlabel('')
		# ax.set_xticklabels([])
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	# ax.set_xticks([0.5,2.5,4.4,6.5,8.5], labels=[-100,-50,0,50,100])
	ax.set_title(f'{title_band_dict[band]}', **timelag_plot_args['ax_title_args'])
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
cax = fig.add_axes([0.92, 0.12, 0.03, 0.75])             # x, y, w, h in [0,1] fig coords
cbar = fig.colorbar(mappable, cax=cax)
cbar.outline.set_linewidth(0.5)
cbar.ax.tick_params(labelsize=6)
cbar.ax.tick_params(labelsize=6)
plt.subplots_adjust(**timelag_plot_args['supplots_adjust'])
plt.suptitle(f'{predictor_area}→{ref_area}', **timelag_plot_args['sup_title_args'])
if save_figs is True:
	plt.savefig(fig_dir + f'additional_questions_bands_on_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_timelag_percents_snr_V1_all_bands_10ms_wsize_avgacrossseeds.pdf', transparent=True, bbox_inches='tight')